In [1]:
# !/usr/bin/env python
# coding: utf-8

import warnings
warnings.simplefilter("ignore")
import plotly.graph_objects as go
import plotly.express as px 
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import recall_score, precision_score
from sklearn.metrics import f1_score, matthews_corrcoef, confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from category_encoders.count import CountEncoder
from training_functions_1 import *

# 0. CARGA DE DATAS.

In [2]:
path = '../outputs/'
df  = get_data(name_sav='df_HOMO_CLASS.sav', path=path)
# df_test = get_data(name_sav='wA_test.sav', path=path)

Es importante decir que se harán uso de algunas funciones presentes en el script __training_functions.py__. Se referirá a ellos con el siguiente estilo: *__estilo__*.

# 1. CONTEXTO DE LA MINICOMPETENCIA.

Estamos ante un problema de clasificación binaria con un evento de desbalanceo importante. 

Se hará una minicompetencia de forma arbitraria, en el sentido de que se considerará varios modelos "clásicos" y uno que es una versión más rápida de un Gradient Boosting. Estos son: `RandomForestClassifier`, `HistGradientBoostingClassifier`, `GradientBoostingClassifier`, `LogisticRegression`.

Los puntos generales a considerar en esta competencia son:
* Se usarán una semilla arbitraria (`seed = 5000`).

* Los modelos se entrenarán con los valores de sus hiperpárametros a como están por default en sus configuraciones desde `scikit-learn`. 

* El mejor modelo será el que tenga un mejor performance en la métrica __f1__ de __cross-validation__, porque, al combinar métricas como el __recall__ y el __precision__, puede interpretarse como una especie de correlación, y entre más cercano a 1, se tendrá tanto un alto __precision__ como un alto __recall__, lo que implica una buena capacidad para clasificar correctamente las muestras positivas y minimizar los falsos positivos y falsos negativos. 

* Aunque el ganador lo dicidirá el __f1__, también se calculará la matriz de confusión de __testeo__, así como los scores de __precision__, __recall__, __coeficiente de correlación de Matthews (m_c)__, de __testeo__ y __cross-validation__, esto con fines informativos. Sin embargo, también pueden ayudar a elegir al modelo ganador en caso de empate.

* También, se calcularán las versiones "micro" de estas métricas, esto para ver vigilar la buena clasificación en ambas clases.

* Existen varios métodos para lidiar con el desbalanceo, entre submuestreos de la clase mayoritaria o sobremuestreos de la clase minoritaria. Aunque un método muy famoso de sobremuestreo es el `SMOTE`, en esta prueba se usará el método de submuestreo `RandomUnderSampler`. Esto es debido a que simplemente, por experiencia, se ha tenido mayor éxito en el pasado usando métodos de submuestreo.

* El subuestreo se hará de tal forma que se tenga un equilibrio total entre la clase minoritaria y la mayoritaria. Esto equivale a tener una estrategia de muestreo igual a 1 (`ratio_balance = 1`).

* Para la codificación de variables categóricas, no se hará uso de ténicas clásicas como el `OneHotEncoder` o el`LabelEncoder`,  si no del `CountEncoder`, esto para evitar la complejización del modelo. El `CountEncoder`, en su versión normalizada, consiste en el porcentaje de representación que tiene cierta clase en una variable categórica.

* No se escalarán las variables numéricas, esto debido a que la mayoría de ellas se mueven en un rango similar de valores.

* Se hará uso de __Pipilines__, es decir, de tuberias de proprocesamiento que contendrán pasos de transformación y de estimación necesarios.

* El uso de __Pipilines__ será especialemnte importante sobretodo en la etapa de __cross-validation k-folds__, para garatizar la repetición de los procesos de imputación, codificación, submuestreo y ajuste del modelo, en cada iteración-__fold__.

* Solo usaremos una cantidad pequeña de __folds__ (`k_folds = 4`).

* El proceso de __cross-validatión__ contempla una estrategia de división estratificada (`StratifiedKFold`). Esto con el fin de que en cada iteración-__fold__ la variable objetivo tenga aproximadamente la misma configuración de porcentajes de clase a como se tiene en el conjunto de __Entrenamiento__. De tal suerte que el proceso de submuetreo será aplicado en cada __fold__ y esto será posible con ayuda los __Pipilines__ ofrecidos por la librería `imbalanced-learn`.

# 2. CONFIGURACIONES GENERALES.

## 2.1. Valores y métricas

Listado de scores de interés:

In [3]:
scores = {'f1': 'f1',
          'f1_micro': 'f1_micro',
          'precision': 'precision',
          'precision_micro': 'precision_micro',
          'recall': 'recall',
          'recall_micro': 'recall_micro',
          'm_c': make_scorer(matthews_corrcoef)}

Se establecen los siguientes valores por default:

In [4]:
seed = 5000
ratio_balance = 1
k_folds = 3
verbose = 10
test_size = 0.25

Se establece el listado de features y la variable objetivo.

In [5]:
features_names =  ['FLAG_OWN_CAR',
                   'FLAG_OWN_REALTY',
                   'NAME_INCOME_TYPE',
                   'NAME_EDUCATION_TYPE',
                   'NAME_FAMILY_STATUS',
                   'NAME_HOUSING_TYPE',
                   'OCCUPATION_TYPE',
                   'FLAG_WORK_PHONE',
                   'FLAG_PHONE',
                   'FLAG_EMAIL',
                   'AGE',
                   'CNT_CHILDREN',
                   'AMT_INCOME_TOTAL',
                   'WORK_YEARS']
objective_name = 'label'

Se obtiene el dataset de features en el conjunto de __Entrenamiento__.

In [6]:
# original_data = df_train.copy()
# data_features = original_data[features_names]

Se obtienen los nombres de variables numéricas y de las categóricas.

In [7]:
# numeric_names

In [8]:
# categorical_names

El establecimiento de los conjuntos de  __entrenamiento__ y __testeo__ se hará dependiendo del estimador que se haya elegido.


Debido al uso de tranformaciones que afectan ya sea solo a variables categóricas o solo variables numéricas, el orden en que aparecen estos grupos de variables en los datasets es importante. Por otro lado, las tranformaciones que se aplican también dependen del estimador que se esté usando, por ejemplo, el `HistGradientBoostingClassifier` no requiere una imputación de valores previa.

El orden en que van aplicandose la tranformaciones va corriendo a la derecha del dataset al grupo de variables al que se termina de aplicar dichas tranformaciones (ver la sección de Notes [aquí](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)
.)

Para lidiar con este problema se hace uso de la función *__get_feature_names_oder__*.

In [9]:
features = df[features_names]
label = df[objective_name]

In [10]:
# numeric_names = list(df.select_dtypes(include=['float64', 'int64']).columns)
# numeric_names = list(set(numeric_names) - set(['ID', 'label']))
# categorical_names = list(df.select_dtypes(include='object').columns)
numeric_names = list(features.select_dtypes(include=['float64', 'int64']).columns)
categorical_names = list(features.select_dtypes(include='object').columns)

In [11]:
categorical_names

['FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'FLAG_EMAIL']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(features,
                                                    label,
                                                    random_state=seed,
                                                    test_size=test_size,
                                                    stratify=label)

In [13]:
feature_names_order = get_feature_names_order(float_names=numeric_names,
                                              categorical_names=categorical_names)

In [14]:
X_train = X_train[feature_names_order]
X_test = X_test[feature_names_order]

In [15]:
X_test

,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,AGE,CNT_CHILDREN,AMT_INCOME_TOTAL,WORK_YEARS
1371,N,Y,Pensioner,Secondary / secondary special,Married,House / apartment,Unidentified,0,0,0,46,1,405000.0,-1000
18,N,Y,Commercial associate,Higher education or Academic degree,Single / not married,House / apartment,Sales staff,0,0,0,51,0,126000.0,6
770,N,N,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,Sales staff,0,0,0,40,1,135000.0,6
278,N,Y,Pensioner,Secondary / secondary special,Single / not married,House / apartment,Unidentified,0,0,0,62,0,225000.0,-1000
1444,Y,Y,Working,Secondary / secondary special,Married,House / apartment,Managers,0,0,0,47,0,315000.0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,Y,Y,Commercial associate,Secondary / secondary special,Married,House / apartment,Core staff,0,1,1,49,1,450000.0,1
834,Y,Y,Working,Higher education or Academic degree,Married,House / apartment,Managers,0,0,0,36,2,180000.0,6
335,N,Y,Pensioner,Secondary / secondary special,Single / not married,House / apartment,Unidentified,0,1,0,56,0,67500.0,-1000
998,N,Y,Pensioner,Secondary / secondary special,Married,House / apartment,Unidentified,0,0,0,64,0,180000.0,-1000


## 2.2 Pipelines de preprocesamiento

Lo siguiente es el __Pipiline__ correspondiente al preprocesamiento de variables numéricas. Este conlleva el utilizar o no técnicas de imputación.
* Para los modelos  `RandomForestClassifier`, `GradientBoostingClassifier`, `LogisticRegression`, se utiliza como imputador la mediana de las variables, esto porque es un estimador robusto ante outliers y para fines de practibilidad. En realidad, en producción se ha utilizado siempre el imputador no parámetrico `KNNImputer`.
* No se utilizará imputador alguno en el caso de `HistGradientBoostingClassifier` ya que este contempla la presencia de missing en su ejecución.

In [16]:
# numeric_transformer = Pipeline(steps=[('imputer', KNNImputer(n_neighbors=3, weights="uniform"))])
# numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])

El siguiente __Pipilene__ corresponde al procesamiento de variables categóricas. Conlleva la codificación `CountEncoder`normalizada y se hace en todos los casos de estimador.

In [17]:
categorical_transformer = Pipeline(steps=[('CountEncoder', CountEncoder(normalize=True))])

In [18]:
# preprocessor = ColumnTransformer(remainder='passthrough',
#                                 transformers=[('categorical',
#                                                 categorical_transformer,
#                                                 cat_names)])

In [19]:
# X_train

In [20]:
# t = preprocessor.fit(X_train[num_names+cat_names])
# t = preprocessor.fit(X_train)

In [21]:
# X_t = t.transform(X_train[num_names+cat_names])

In [22]:
# X_train

In [23]:
# X_train[num_names+cat_names]

In [24]:
# X_t

Estos prepocesamientos para data numérica y categórca dan lugar un objeto __preprocesor__ de la siguiente manera:
```
preprocessor = ColumnTransformer(remainder='passthrough',
                                transformers=[('categorical',
                                                categorical_transformer,
                                                categorical_names)])
```

Lo anterior se reflejará al utilizar la función *__get_preprocessor__*.

## 2.3 Pipeline de Tranformación

El pipeline __transform__ contiene  los procesos de preprocesamiento (ver subsección 2.2), la técnica de submuestreo de la clase mayoritaria y el estimador que se utilizará. Está dado por:
```
transform = Pipeline(steps=[('processing',
                              preprocessor),
                            ('RandomUnderSampler',
                              RandomUnderSampler(random_state=seed,
                                                 sampling_strategy=ratio_balance)),
                            ('estimator',
                              model)])
```

# 3. COMPETENCIA DE MODELOS

In [25]:
models = [
          ('HistGradientBoostingClassifier', HistGradientBoostingClassifier(random_state=seed)),
          ('RandomForest', RandomForestClassifier(random_state=seed)),
          ('GradientBoosting', GradientBoostingClassifier(random_state=seed)),
          ('LogisticRegression', LogisticRegression(random_state=seed))
         ]

In [26]:
pr_curves = []
f1_testeo_scores = []

# ITERAR SOBRE CADA MODELO Y CALCULAR LA CURVA PR
for model_name, model in models:
    print(f'NOMBRE: {model_name}')
    # CONFIGURACIONES GENERALES. ###########

    preprocessor = get_preprocessor(float_names=numeric_names,
                                    categorical_names=categorical_names)
    transform = Pipeline(steps=[('processing',
                                  preprocessor),
                                ('RandomUnderSampler',
                                  RandomUnderSampler(random_state=seed,
                                                     sampling_strategy=ratio_balance)),
                                ('estimator',
                                  model)])

    # AJUSTE DEL MODELO. ###################
    clf = transform.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    f1_testeo = f1_score(y_test, y_pred)
    f1_testeo_scores.append(f1_testeo)

    # OBTENER PROBABILIDADES ESTIMADAS #########
    probas_pred = clf.predict_proba(X_test)[:, 1]

    # CALCULAR LA CURVA PR Y EL ÁREA BAJO LA CURVA (AUC-PR) ##########
    precision, recall, _ = precision_recall_curve(y_test, probas_pred)
    auc_pr = auc(recall, precision)

    # GUARDAR LA CURVA PR Y ETIQUETA EN LA LISTA ####################
    pr_curves.append((recall, precision, f'{model_name} (AUC = {auc_pr:.2f})'))

NOMBRE: HistGradientBoostingClassifier
NOMBRE: RandomForest
NOMBRE: GradientBoosting
NOMBRE: LogisticRegression


In [27]:
# CREAR LA FIGURA
fig = go.Figure()
colors = px.colors.qualitative.Vivid

# AGREGAR TODAS LAS CURVAS PR
for i, (recall, precision, label) in enumerate(pr_curves):
    fig.add_trace(go.Scatter(x=recall, y=precision, mode='lines', name=label, line=dict(color=colors[i])))

In [28]:
# MOSTRAR LAS CURVAS
# CONFIGURAR EL DISEÑO DE LAS GRÁFICAS
fig.update_layout(
    title='Curvas PR de Modelos',
    xaxis=dict(title='Recall'),
    yaxis=dict(title='Precision'),
    legend=dict(x=0.05, y=0.05),
    showlegend=True,
    template='plotly_white'
)
# fig.show()

In [29]:
fig.show()

Se observa que el `HistGradientBoostingClassifier` tiene una mayor area bajo la curva PR. Veamos cómo se comporta el F1 de testeo.

In [30]:
# LISTA DE MODELOS Y F1-SCORES
model_names = [model[0] for model in models]
colors = px.colors.qualitative.Vivid

# CREAR UNA GRÁFICA DE BARRAS
fig = go.Figure(data=[go.Bar(x=model_names, y=f1_testeo_scores, marker_color=colors)])

# PERSONALIZAR Y MOSTRAR LA GRÁFICA
fig.update_layout(title='Comparativa de Modelos',
                  xaxis_title='Modelos',
                  yaxis_title='F1-score',
                  yaxis=dict(range=[0, 1]))

fig.show()

Observamos que el `HistGradientBoostingClassifier` y `Random Forest` tienen los mejores F1 de testeo. Vamos a tomar a estos dos modelos como "finalistas" y vemos como se comportan sus métricas de testeo y cross-validation. Por medio del cross-validation, veremos que tan estables son estos modelos.

# 4. ENTRENAMIENTO DE HISTOGRAM GRADIENT BOOSTING.

Entrenemos el estimador `HistGradientBoostingClassifier`.

In [31]:
model_name = 'HistGradientBoostingClassifier'

Se muestra el orden en que se aplicarán los procesos de tranformación en el __Pipiline general (transform)__ :
1. Preprocesamiento de variables (codificación).
2. Submuestro.
3. Ajuste de modelo.

In [32]:
preprocessor = get_preprocessor(float_names=numeric_names,
                                categorical_names=categorical_names)
transform = Pipeline(steps=[("processing", preprocessor),
                            ("RandomUnderSampler", RandomUnderSampler(random_state=seed,
                                                                      sampling_strategy=ratio_balance)),
                            ("estimator",  HistGradientBoostingClassifier(random_state=seed))])
transform

Pipeline(steps=[('processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  Pipeline(steps=[('CountEncoder',
                                                                   CountEncoder(combine_min_nan_groups=True,
                                                                                normalize=True))]),
                                                  ['FLAG_OWN_CAR',
                                                   'FLAG_OWN_REALTY',
                                                   'NAME_INCOME_TYPE',
                                                   'NAME_EDUCATION_TYPE',
                                                   'NAME_FAMILY_STATUS',
                                                   'NAME_HOUSING_TYPE',
                                                   'OCCUPATION_TYPE',
                                                   'FLAG_WORK_PHONE',
                                                   'FLAG_PHONE',
                                                   'FLAG_EMAIL'])])),
                ('RandomUnderSampler',
                 RandomUnderSampler(random_state=5000, sampling_strategy=1)),
                ('estimator',
                 HistGradientBoostingClassifier(random_state=5000))])

## 4. 1 Ajuste del modelo para obtener métricas de testeo.

In [33]:
clf = transform.fit(X_train, y_train)

In [34]:
y_pred = clf.predict(X_test)

A continuación se exhiben las métricas de __testeo__, solo con fines informativos.

In [35]:
f_1_testeo = f1_score(y_test, y_pred)
f_1_testeo_micro = f1_score(y_test, y_pred, average='micro')
precision_testeo = precision_score(y_test, y_pred)
precision_micro_testeo = precision_score(y_test, y_pred, average='micro')
recall_testeo = recall_score(y_test, y_pred)
recall_micro_testeo = recall_score(y_test, y_pred, average='micro')
m_c_testeo = matthews_corrcoef(y_test, y_pred)
cm_testeo = confusion_matrix(y_test, y_pred)

In [36]:
print('f1-testeo:', round(f_1_testeo, 2))
print('f1-micro-testeo:', round(f_1_testeo_micro, 2))
print('precision-testeo:', round(precision_testeo, 2))
print('precision-micro-testeo:', round(precision_micro_testeo, 2))
print('recall-testo:', round(recall_testeo, 2))
print('recall-micro-testo:', round(recall_micro_testeo, 2))
print('m_c-testeo:', round(m_c_testeo, 2))
print('matriz de confusión de testeo', cm_testeo)

f1-testeo: 0.29
f1-micro-testeo: 0.59
precision-testeo: 0.18
precision-micro-testeo: 0.59
recall-testo: 0.73
recall-micro-testo: 0.59
m_c-testeo: 0.19
matriz de confusión de testeo [[189 143]
 [ 12  32]]


## 4.2 Cross-validation.

In [37]:
c_v = StratifiedKFold(n_splits=k_folds,
                      shuffle=True,
                      random_state=seed)

In [38]:
cv_results = cross_validate(transform,
                            X_train,
                            y_train,
                            cv=c_v,
                            scoring=scores,
                            verbose=10)

[CV] START .....................................................................
[CV] END  f1: (test=0.192) f1_micro: (test=0.574) m_c: (test=0.016) precision: (test=0.123) precision_micro: (test=0.574) recall: (test=0.432) recall_micro: (test=0.574) total time=   0.2s
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.2s


[CV] END  f1: (test=0.237) f1_micro: (test=0.555) m_c: (test=0.098) precision: (test=0.148) precision_micro: (test=0.555) recall: (test=0.605) recall_micro: (test=0.555) total time=   0.2s
[CV] START .....................................................................
[CV] END  f1: (test=0.215) f1_micro: (test=0.493) m_c: (test=0.046) precision: (test=0.131) precision_micro: (test=0.493) recall: (test=0.591) recall_micro: (test=0.493) total time=   0.2s


Se obtienen las métricas de __cross-validation__.

In [39]:
print('f1-cv:', round(cv_results['test_f1'].mean(), 2))
print('f1-micro-cv:', round(cv_results['test_f1_micro'].mean(), 2))
print('precision-cv:', round(cv_results['test_precision'].mean(), 2))
print('precision-micro-cv:', round(cv_results['test_precision_micro'].mean(), 2))
print('recall-cv:', round(cv_results['test_recall'].mean(), 2))
print('recall-micro-cv:', round(cv_results['test_recall_micro'].mean(), 2))
print('m_c-cv:', round(cv_results['test_m_c'].mean(), 2))

f1-cv: 0.21
f1-micro-cv: 0.54
precision-cv: 0.13
precision-micro-cv: 0.54
recall-cv: 0.54
recall-micro-cv: 0.54
m_c-cv: 0.05


# 5. ENTRENAMIENTO DE RANDOM FOREST.

Entrenemos el estimador `RandomForestClassifier`.

In [40]:
model_name = 'RandomForestClassifier'

Se obtienen las versiones definitivas de los conjuntos de __Entrenamiento__ y de __Testeo__.

Se muestra el orden en que se aplicarán los procesos de tranformación en el __Pipiline general (transform)__:
1. Preprocesamiento de variables (imputación y codificación).
2. Submuestro.
3. Ajuste de modelo.

In [41]:
preprocessor = get_preprocessor(float_names=numeric_names,
                                categorical_names=categorical_names)
transform = Pipeline(steps=[("processing", preprocessor),
                            ("RandomUnderSampler", RandomUnderSampler(random_state=seed,
                                                                      sampling_strategy=ratio_balance)),
                            ("estimator",  RandomForestClassifier(random_state=seed))])
transform

Pipeline(steps=[('processing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  Pipeline(steps=[('CountEncoder',
                                                                   CountEncoder(combine_min_nan_groups=True,
                                                                                normalize=True))]),
                                                  ['FLAG_OWN_CAR',
                                                   'FLAG_OWN_REALTY',
                                                   'NAME_INCOME_TYPE',
                                                   'NAME_EDUCATION_TYPE',
                                                   'NAME_FAMILY_STATUS',
                                                   'NAME_HOUSING_TYPE',
                                                   'OCCUPATION_TYPE',
                                                   'FLAG_WORK_PHONE',
                                                   'FLAG_PHONE',
                                                   'FLAG_EMAIL'])])),
                ('RandomUnderSampler',
                 RandomUnderSampler(random_state=5000, sampling_strategy=1)),
                ('estimator', RandomForestClassifier(random_state=5000))])

In [42]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('categorical',
                                 Pipeline(steps=[('CountEncoder',
                                                  CountEncoder(combine_min_nan_groups=True,
                                                               normalize=True))]),
                                 ['FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
                                  'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
                                  'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE',
                                  'OCCUPATION_TYPE', 'FLAG_WORK_PHONE',
                                  'FLAG_PHONE', 'FLAG_EMAIL'])])

## 5. 1 Ajuste del modelo para obtener métricas de testeo.

In [43]:
clf = transform.fit(X_train, y_train)

In [44]:
y_pred = clf.predict(X_test)

A continuación se exhiben las métricas de __testeo__, solo con fines informativos.

In [45]:
f_1_testeo = f1_score(y_test, y_pred)
f_1_testeo_micro = f1_score(y_test, y_pred, average='micro')
precision_testeo = precision_score(y_test, y_pred)
precision_micro_testeo = precision_score(y_test, y_pred, average='micro')
recall_testeo = recall_score(y_test, y_pred)
recall_micro_testeo = recall_score(y_test, y_pred, average='micro')
m_c_testeo = matthews_corrcoef(y_test, y_pred)
cm_testeo = confusion_matrix(y_test, y_pred)

In [46]:
print('f1-testeo:', round(f_1_testeo, 2))
print('f1-micro-testeo:', round(f_1_testeo_micro, 2))
print('precision-testeo:', round(precision_testeo, 2))
print('precision-micro-testeo:', round(precision_micro_testeo, 2))
print('recall-testo:', round(recall_testeo, 2))
print('recall-micro-testo:', round(recall_micro_testeo, 2))
print('m_c-testeo:', round(m_c_testeo, 2))
print('matriz de confusión de testeo', cm_testeo)

f1-testeo: 0.33
f1-micro-testeo: 0.65
precision-testeo: 0.21
precision-micro-testeo: 0.65
recall-testo: 0.73
recall-micro-testo: 0.65
m_c-testeo: 0.24
matriz de confusión de testeo [[214 118]
 [ 12  32]]


## 5.2 Cross-validation.

In [47]:
c_v = StratifiedKFold(n_splits=k_folds,
                      shuffle=True,
                      random_state=seed)

In [48]:
cv_results = cross_validate(transform,
                            X_train,
                            y_train,
                            cv=c_v,
                            scoring=scores,
                            verbose=10)

[CV] START .....................................................................
[CV] END  f1: (test=0.238) f1_micro: (test=0.609) m_c: (test=0.094) precision: (test=0.154) precision_micro: (test=0.609) recall: (test=0.523) recall_micro: (test=0.609) total time=   0.2s
[CV] START .....................................................................


[Parallel(n_jobs=1)]: Done   1 tasks      | elapsed:    0.2s


[CV] END  f1: (test=0.201) f1_micro: (test=0.597) m_c: (test=0.039) precision: (test=0.130) precision_micro: (test=0.597) recall: (test=0.442) recall_micro: (test=0.597) total time=   0.2s
[CV] START .....................................................................
[CV] END  f1: (test=0.274) f1_micro: (test=0.533) m_c: (test=0.164) precision: (test=0.168) precision_micro: (test=0.533) recall: (test=0.750) recall_micro: (test=0.533) total time=   0.2s


Se obtienen las métricas de __cross-validation__.

In [49]:
print('f1-cv:', round(cv_results['test_f1'].mean(), 2))
print('f1-micro-cv:', round(cv_results['test_f1_micro'].mean(), 2))
print('precision-cv:', round(cv_results['test_precision'].mean(), 2))
print('precision-micro-cv:', round(cv_results['test_precision_micro'].mean(), 2))
print('recall-cv:', round(cv_results['test_recall'].mean(), 2))
print('recall-micro-cv:', round(cv_results['test_recall_micro'].mean(), 2))
print('m_c-cv:', round(cv_results['test_m_c'].mean(), 2))

f1-cv: 0.24
f1-micro-cv: 0.58
precision-cv: 0.15
precision-micro-cv: 0.58
recall-cv: 0.57
recall-micro-cv: 0.58
m_c-cv: 0.1


# 6. MODELO GANADOR.

El `RandomForestClassifier` es el modelo ganador en todas las métricas, en particular el __f1 de validation__ con 0.24 frente a 0.21 del `HistogramGradientBoosting`. 